In [9]:
from doe_sampling import Space, Lhs, Path
import numpy as np
import pandas as pd
import logging

In [10]:


#def write_lhs_file(variable_ranges, samples, path, labels):
def write_lhs_file(doe_variables, samples, path):
    variable_constants = []
    variable_variables = []
    variable_ranges = doe_variables.values()
    labels=','.join(['ID'] + [key for key in doe_variables.keys()])
    
    for var in variable_ranges:
        if not isinstance(var, tuple):
            variable_constants.append(var)
        elif var[0] == var[1]:
            variable_constants.append(var[0])
        else:
            variable_variables.append(var)

    space = Space(variable_variables)
    lhs = Lhs(criterion="maximin", iterations=10000)
    x = lhs.generate(space.dimensions, samples)

    for xi in x:
        [xi.append(var) for var in variable_constants]

    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w') as f:
        f.write(f'{labels}\n')
        for i in range(len(x)):
            f.write(f'{i+1}')
            for j in range(len(np.array(x)[i,:])):
                f.write(f', {np.array(x)[i,j]}')
            f.write('\n')
    print(path)

In [11]:
SAMPLES = 30
DATASET_ROOT = f'/home/ppiper/ihtc_repository/data/doe_30/'
DOEFILE = f'{DATASET_ROOT}doe_lhs.txt'

DOE_VARIABLES = {
                #'T0in' : (285.0, 1115.0),
                 'Thickness' : (0.001, 0.010),
                #'p0in': (0.5e6, 1e6),
                 'CP3_y': (-0.01255805, 0.0),
                 }

OTHER_PARAMS = {
                'Nx': 210,
                'Ny': 330,
                'tol': 1e-8,
                'cores': None,
                'inflationRate': 1.0015,
                'baselineCP' : '/home/ppiper/Dropbox/local/ihtc_repository/src/baselineCP.txt',
                'metal': 'AISI406',
                'itmaxSU2': 4_000,
                'rootfile': DATASET_ROOT, 
}

In [14]:
write_lhs_file(DOE_VARIABLES, SAMPLES, DOEFILE)

/home/ppiper/ihtc_repository/data/doe_30/doe_lhs.txt


In [27]:
import subprocess
from subprocess import PIPE

subprocess.call(["./multirun.sh", f"/home/ppiper/ihtc_repository/src/multiple_run"],  stdin=PIPE, stderr=PIPE, stdout=PIPE, shell=True)

0

In [22]:
import pathlib
pathlib.Path('../data/doe_30').resolve()

PosixPath('/home/ppiper/ihtc_repository/data/doe_30')

In [39]:
import configparser
class CaseConfigParser(configparser.ConfigParser):
    def optionxform(self, optionstr):
        return optionstr

config = CaseConfigParser()
config.read('/home/ppiper/ihtc_repository/src/doe_30.cfg')

['/home/ppiper/ihtc_repository/src/doe_30.cfg']

In [42]:
for each_section in config.sections():
    for (each_key, each_val) in config.items(each_section):
        value = pathlib.Path(config[each_section][each_key])
        if value.is_file() or value.is_dir():
            value = value.resolve()
            print(value)

/home/ppiper/ihtc_repository/src/models_parametric_shape.py
/home/ppiper/ihtc_repository/data/doe_30/doe_lhs.txt
/home/ppiper/ihtc_repository/src/gmsh
/home/ppiper/ihtc_repository/src/SU2_CFD
/home/ppiper/ihtc_repository/src/baselineCP.txt
/home/ppiper/ihtc_repository/src/eulerQ1D
/home/ppiper/ihtc_repository/src/baselineCP.txt
/home/ppiper/ihtc_repository/src/multiple_run
/home/ppiper/ihtc_repository/src/run_master.py
/home/ppiper/ihtc_repository/data/doe_30


In [47]:
for each_section in config.sections():
    for (each_key, each_val) in config.items(each_section):
        value = pathlib.Path(config[each_section][each_key])
        if value.is_file() or value.is_dir():
            config[each_section][each_key] = f"{value.resolve()}"
        print(config[each_section][each_key])

/home/ppiper/ihtc_repository/src/models_parametric_shape.py
/home/ppiper/ihtc_repository/data/doe_30/doe_lhs.txt
210
330
1e-16
None
1.0015
AISI406
3_000
/home/ppiper/ihtc_repository/src/gmsh
/home/ppiper/ihtc_repository/src/SU2_CFD
/home/ppiper/ihtc_repository/src/baselineCP.txt
600.0
8.0e5
1
400
/home/ppiper/ihtc_repository/src/eulerQ1D
/home/ppiper/ihtc_repository/src/baselineCP.txt
600.0
8.0e5
15
/home/ppiper/ihtc_repository/src/multiple_run
/home/ppiper/ihtc_repository/src/run_master.py
/home/ppiper/ihtc_repository/data/doe_30
